# [IAPR][iapr]: Project


**Group ID:** 32

**Author 1 (sciper):** Ghali CHRAIBI (262251)  
**Author 2 (sciper):** Yann Yasser HADDAD (272292)   
**Author 3 (sciper):** Julien BERGER (247179)   

**Release date:** 07.05.2021  
**Due date:** 03.06.2021 (23h59)


## Important notes

The lab assignments are designed to teach practical implementation of the topics presented during class as well as preparation for the final project, which is a practical project which ties together the topics of the course. 

As such, in the lab assignments/final project, unless otherwise specified, you may, if you choose, use external functions from image processing/ML libraries like opencv and sklearn as long as there is sufficient explanation in the lab report. For example, you do not need to implement your own edge detector, etc.

**! Before handling back the notebook !** rerun the notebook from scratch `Kernel` > `Restart & Run All`


[iapr]: https://github.com/LTS5/iapr

---
## 0. Introduction

An anonymous researcher that we will name Lann Yecun is convinced that the MNIST dataset still has great potential. He decides to create a playing card game based on MNIST digits and different figures. The game uses a standard 52 card deck which is composed of four French suits/colours: clubs (&#9827;), diamonds (&#9830;), hearts (&#9829;) and spades (&#9824;). Each suit includes 10 digit cards (from 0 to 9) and 3 figures (Jack-J, Queen-Q, and King-K). Here is an example of the 13 spade cards with their name.


<img src="data/media/example_cards.png">


We can find the same arrangement of cards for the clubs, diamonds, and hearts. 


## 1. Rules


### 1.1 Standard

The rules are based on the simple battle card game. The goal of the game is to win as many points as possible. Each turn, the 4 players play a card in front of them. As displayed in the example below. The rules are the following:

- The cards are ranked in the following order : **0 < 1 < 2 < 3 < 4 < 5 < 6 < 7 < 8 < 9 < J < Q < K**.
- The player with the highest-ranked card wins the round and obtains **1 point**. 
- If the highest-ranked card is the same for multiple players we call it a draw and all winners get **1 points**. 
- In this configuration, we **do not** take into account the suits. The game only rely on the card ranks. 
- The game lasts 13 rounds. After the last round, the winner is the player that has the largest number of points. 
- In the example below Player 1 wins the round with his Queen ( 0 < 8 < J < **Q**).

If two or more players have the same number of points they share the victory.

### 1.2 Advanced

The advanced rules take into account the suits. 

- At the beginning of **each round** a random player is designated as the **dealer**. The dealer places a green token with the letter *D* next to him (player 1 in the example below).
- Only the cards that belong to the same suit as the one of the dealer are considered valid. In the example below, only Player 4 is competing with Player 1 as spade was selected by the dealer (e.i., Player 1). Player 2 and 3 are out for this round. Player 1 wins the round and **1 point** with the Queen ( 0&#9824; < **Q&#9824;**).
- There cannot be any draw between the players as they are not any card duplicates.
- We use the same system as the standard method to count the points.


<img src="data/media/example_round.jpg">


### 1.3 Notes

- The orientation of the card is linked to the position of the player around the table. For instance, to read the card of the 3rd player you will have to rotate it by 180°.
- The **digits** always **face** the players around the table. The figures can have random orientations.
- Player 1 **always** seats south of the table. The players are **always** ordered counter-clockwise as in the example. 
- The dealers can change between the rounds and games.
- Some cards **might** apear multiple times per game.
- Pictures are always taken from rougthly the same altitude.
- The digits from the training set **would not** be the same as the one of the testing set.

---
## 2. Data

You will be given the images of 7 games that were played ([download link](https://drive.google.com/drive/folders/1fEy27wnJsUJPRsEEomzoAtP56s-7HFtk?usp=sharing)). The data are composed of:
   - 7 folder named after the games (game1 to game7).
   - Each game includes 13 ordered images (1st to 13th round).
   - Each game includes a csv file with the ground truth of the game. The first row list the players (P1 to P4) as well as the dealer (D). The following rows represent the rounds (1 to 13). We represent the card played with 2 character as $AB$ where $A \in [0-9, J, Q, K]$ is the rank of the card and $B \in [C, D, H, S]$ is the suit. For example, QS means "(Q)ueen of (S)pade" and 0D means "(0) of (D)iamond". The dealer is represented by the ID of the player (e.g. P1 -> 1).
   
You are free to use external datasets such as the original MNIST train set that you used in lab 3.

---
## 3. Your Tasks

Your task is to ready yourself for the final evaluation. The day of the exam we will give you a new folder with a new game. ! The digits on the cards **differ** from the one of the traning set. When given a new data folder with 13 images your should be able to:

**Task 0**
   - Plot an overlay for each round image that shows your detections and classification. You can for example plot bounding boxes around the cards/dealer token and add a text overlay with the name of the classes.

**Task 1**
   - (a) Predict the **rank** of the card played by each player at each round (Standard rules).
   - (b) Predict the **number of points** of each player according to **Standard** rules
 
**Task 2**
   - (a) Detect which player is the selected **dealer** for each round.
   - (b) Predict the **rank** and the **suit** of the card played by each player at each round (Advanced rules).
   - (c) Predict the **number of points** of each player according to **Advanced** rules

---

**Before the exam (until 03.06.21 at 23h59)**
   - Create a zipped folder named **group_xx.zip** that you upload on moodle (xx being your group number).
   - Include a **runnable** code (Jupyter Notebook and external files) and your presentation in the zip folder.
   
**The day of the exam (04.06.21)**
   - You will be given a **new folder** with 13 images (rounds) and but **no ground truth** (csv file).
   - We will ask you to run your pipeline in **realtime** and to send us your prediction of task 1 and 2 that you obtain with the function **print_results**. 
   - On our side we will compute the perfomance of your classification algorithm. 
   - To evaluate your method we will use the **evaluate_game** function presented below. To understand how the provided functions work please read the documentation of the functions in **utils.py**.
   - **Please make sure your function returns the proper data format to avoid points penalty the day of the exam**. 

## Imports & Constants
---

In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import skimage.io

import torch
from torch import nn
from torch.utils import data

import torchvision.transforms as transforms

from MNISTDataset import MNISTDataset
from CNN import CNN

from utils import *
from MNIST_utils import extract_data, extract_labels
from ML_pipeline import augment_data, train_loop, test_loop, predict

In [23]:
NB_CLASS_RANK = 13
NB_CLASS_SUIT = 4

TRAIN_RANK_MODEL = True
SAVE_RANK_MODEL = True
TRAIN_SUIT_MODEL = True
SAVE_SUIT_MODEL = True

## Data loading
---

In [3]:
train_data = 'data/train_games'
train_game_count = 7
game_round_count = 13

dict_data = {}

for i in range(1, train_game_count+1):
    # For each game, store the path of the csv and the image of each round
    dict_data[f'game{i}'] = {}
    dict_data[f'game{i}']['url'] = train_data + f'/game{i}'
    dict_data[f'game{i}']['csv'] = train_data + f'/game{i}.csv'
    
    for j in range(1, game_round_count+1):
        dict_data[f'game{i}'][f'round{j}'] = skimage.io.imread(train_data + f'/game{i}/{j}.jpg')

## Rank detection model
---

### Load MNIST data

In [4]:
image_shape = (28, 28)
train_set_size = 60000
test_set_size = 10000

mnist_folder = os.path.join('data', 'MNIST')

mnist_train_images_path = os.path.join(mnist_folder, 'train-images-idx3-ubyte.gz')
mnist_train_labels_path = os.path.join(mnist_folder, 'train-labels-idx1-ubyte.gz')
mnist_test_images_path = os.path.join(mnist_folder, 't10k-images-idx3-ubyte.gz')
mnist_test_labels_path = os.path.join(mnist_folder, 't10k-labels-idx1-ubyte.gz')

mnist_train_images = extract_data(mnist_train_images_path, image_shape, train_set_size)
mnist_test_images = extract_data(mnist_test_images_path, image_shape, test_set_size)
mnist_train_labels = extract_labels(mnist_train_labels_path, train_set_size)
mnist_test_labels = extract_labels(mnist_test_labels_path, test_set_size)

### Load segmented ranks

In [5]:
filepath_segmented_rank = os.path.join('data', 'segmented_data', 'all_games_ranks.pickle')
df_segmented_rank, df_segmented_numbers, df_segmented_figures = load_segmented_rank(filepath_segmented_rank)

### Data augmentation

In [6]:
transform_figure = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=25, scale=(0.7, 1), shear=25),
        transforms.ToTensor(),
    ]
)

transform_mnist = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=25, scale=(0.8, 1.1), shear=20),
        transforms.ToTensor(),
    ]
)

In [7]:
figures_label = {'J': 10, 'Q': 11, 'K': 12}

train_augmented_figs, train_augmented_figs_labels, val_augmented_figs, \
        val_augmented_figs_labels, test_figs, test_figs_labels = \
            get_train_val_test_figures(df_segmented_figures, figures_label, transform_figure)

train_augmented_mnist = augment_data(mnist_train_images, transform_mnist)

### Create DataLoaders

In [8]:
# Parameters for the DataLoaders
params = {'batch_size': 128,
          'shuffle': True}

In [32]:
# Creation of a train/test dataset & dataloader
train_merged_images = np.concatenate((mnist_train_images, train_augmented_mnist, train_augmented_figs))
train_merged_labels = np.concatenate((mnist_train_labels, mnist_train_labels, train_augmented_figs_labels))

val_merged_images = np.concatenate((mnist_test_images, val_augmented_figs))
val_merged_labels = np.concatenate((mnist_test_labels, val_augmented_figs_labels)) 

ds_train_rank = MNISTDataset(train_merged_images, train_merged_labels, transform=transforms.ToTensor())
dl_train_rank = data.DataLoader(ds_train_rank, **params)

ds_val_rank = MNISTDataset(val_merged_images, val_merged_labels, transform=transforms.ToTensor())
dl_val_rank = data.DataLoader(ds_val_rank, **params)

In [33]:
segmented_numbers = np.array([v for v in df_segmented_numbers.image.apply(img_as_ubyte)])

ds_test_numbers = MNISTDataset(segmented_numbers, df_segmented_numbers['rank'].to_numpy().astype(np.int64),
                               transform=transforms.ToTensor())
dl_test_numbers = data.DataLoader(ds_test_numbers, **params)

In [34]:
ds_test_figs = MNISTDataset(test_figs, test_figs_labels.astype(np.int64), transform=transforms.ToTensor())
dl_test_figs = data.DataLoader(ds_test_figs, **params)

### Build & Train the model 

In [35]:
if TRAIN_RANK_MODEL:
    rank_model = CNN()

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer
    learning_rate = 1e-3
    opt = torch.optim.Adam(rank_model.parameters(), lr=learning_rate)

    epochs = 7

    # Train & test our model until convergence
    for e in range(epochs):
        print(f"Epoch {e+1}\n-------------------------------")
        train_loop(dl_train_rank, rank_model, criterion, opt)
        test_loop(dl_val_rank, rank_model, criterion)

Epoch 1
-------------------------------
It 1055/1055:	Loss train: 0.04867, Accuracy train: 97.56%%
Test Error:
	Avg loss: 0.00037, Accuracy: 98.36%

Epoch 2
-------------------------------
It 1055/1055:	Loss train: 0.08209, Accuracy train: 97.56%%
Test Error:
	Avg loss: 0.00049, Accuracy: 98.08%

Epoch 3
-------------------------------
It 1055/1055:	Loss train: 0.00184, Accuracy train: 100.00%
Test Error:
	Avg loss: 0.00076, Accuracy: 97.70%

Epoch 4
-------------------------------
It 1055/1055:	Loss train: 0.00812, Accuracy train: 100.00%
Test Error:
	Avg loss: 0.00072, Accuracy: 97.91%

Epoch 5
-------------------------------
It 1055/1055:	Loss train: 0.03904, Accuracy train: 97.56%%
Test Error:
	Avg loss: 0.00075, Accuracy: 98.04%

Epoch 6
-------------------------------
It 1055/1055:	Loss train: 0.09817, Accuracy train: 98.78%%
Test Error:
	Avg loss: 0.00066, Accuracy: 98.47%

Epoch 7
-------------------------------
It 1055/1055:	Loss train: 0.06091, Accuracy train: 98.78%%
Test Er

### Test the model

In [36]:
if TRAIN_RANK_MODEL:
    test_loop(dl_test_numbers, rank_model, criterion)

Test Error:
	Avg loss: 0.01094, Accuracy: 82.08%



In [37]:
if TRAIN_RANK_MODEL:
    test_loop(dl_test_figs, rank_model, criterion)

Test Error:
	Avg loss: 0.02205, Accuracy: 94.44%



### Save the model

In [38]:
if SAVE_RANK_MODEL:
    models_dir = os.path.join('data', 'models')

    if not os.path.isdir(models_dir):
        os.mkdir(models_dir)
        
    torch.save(rank_model.state_dict(), os.path.join(models_dir, "model_rank.pt"))

In [40]:
### TODO do something with this TODO ###
predict(segmented_numbers, NB_CLASS_RANK, CNN, "data/models/model_rank.pt")

tensor([ 8,  0,  5,  9, 10,  3,  2,  3,  8,  7,  4,  0,  6,  3,  2,  8,  8,  3,
         4,  7,  2,  6,  9,  7, 10,  5,  5,  1,  7,  6, 10,  4,  2,  6,  8,  2,
         1,  5,  0,  2,  2,  7, 10,  6,  8,  4,  8,  2,  5,  1,  5,  0,  0,  3,
         4,  1,  0,  7,  2,  0,  2,  1,  6,  4,  8,  2,  1,  8,  0,  3,  9,  5,
         5,  5,  3,  6,  6,  3,  7,  6,  0,  1,  1,  7,  6,  3,  2,  3,  7,  4,
         2,  6,  9,  7,  0,  8,  3,  0,  0,  5,  1,  3,  4,  0, 10,  2,  5,  2,
         7,  6,  5,  6,  5,  4,  2,  8,  8,  8,  8,  3,  2,  6,  3,  9,  3,  5,
         7,  7,  3,  8,  8,  8,  4,  2,  1,  0, 10,  0,  8,  7,  7,  0,  1,  2,
         0,  5,  2,  5,  4,  5,  2,  8,  4,  6,  3,  5,  6,  6,  7,  2,  2,  9,
         0,  2,  3,  7,  6,  8,  4,  7,  6,  3,  5,  5,  3,  6,  6,  3,  8,  5,
         1,  5,  2,  8,  4,  3,  4,  7,  5,  0, 10,  8,  7,  0,  1, 10,  6,  0,
         1,  7,  5,  4,  0,  7,  6, 10,  8,  5,  5,  3,  2,  1,  2,  9, 10,  6,
         2,  7,  3,  1,  0,  2,  3,  1, 

## Suit detection model
---

### Load segmented ranks

In [19]:
filepath_suits = os.path.join('data', 'segmented_data', 'all_games_suits.pickle')

segmented_suits = pd.read_pickle(filepath_suits)
segmented_suits_values = segmented_suits.values
imgs = [v[1] for v in segmented_suits_values]
ranks = [v[0][0] for v in segmented_suits_values]
suits = [v[0][1] for v in segmented_suits_values]

df_suits = pd.DataFrame({"rank": ranks, "suit": suits, "image": imgs})

### Data augmentation

In [20]:
transform_suit = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomAffine(degrees=25, scale=(0.7, 1), shear=25),
        transforms.ToTensor(),
    ]
)

In [21]:
suits_labels = {'C': 0, 'D': 1, 'H': 2, 'S': 3}

train_augmented_suits, train_augmented_suits_labels, val_augmented_suits, val_augmented_suits_labels, \
            test_suits, test_suits_labels = \
            get_train_val_test_suits(df_suits, suits_labels, transform_suit)

### Create DataLoaders

In [22]:
# Creation of a train/test dataset & dataloader 
ds_train_suit = MNISTDataset(train_augmented_suits, train_augmented_suits_labels, transform=transforms.ToTensor())
dl_train_suit = data.DataLoader(ds_train_suit, **params)

ds_val_suit = MNISTDataset(val_augmented_suits, val_augmented_suits_labels, transform=transforms.ToTensor())
dl_val_suit = data.DataLoader(ds_val_suit, **params)

ds_test_suit = MNISTDataset(test_suits, test_suits_labels, transform=transforms.ToTensor())
dl_test_suit = data.DataLoader(ds_test_suit, **params)

### Build & Train the model 

In [24]:
if TRAIN_SUIT_MODEL:
    model = CNN(nb_classes=4)

    # Loss function
    criterion = nn.CrossEntropyLoss()

    # Optimizer
    learning_rate = 1e-3
    opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

    epochs = 5

    # Train & test our model until convergence
    for e in range(epochs):
        print(f"Epoch {e+1}\n-------------------------------")
        train_loop(dl_train_suit, model, criterion, opt)
        test_loop(dl_val_suit, model, criterion)

Epoch 1
-------------------------------
It 30/30:	Loss train: 0.26477, Accuracy train: 95.29%
Test Error:
	Avg loss: 0.00198, Accuracy: 91.35%

Epoch 2
-------------------------------
It 30/30:	Loss train: 0.13751, Accuracy train: 97.65%%
Test Error:
	Avg loss: 0.00032, Accuracy: 99.19%

Epoch 3
-------------------------------
It 30/30:	Loss train: 0.04598, Accuracy train: 98.82%%
Test Error:
	Avg loss: 0.00014, Accuracy: 99.73%

Epoch 4
-------------------------------
It 30/30:	Loss train: 0.08945, Accuracy train: 97.65%%
Test Error:
	Avg loss: 0.00013, Accuracy: 100.00%

Epoch 5
-------------------------------
It 30/30:	Loss train: 0.02843, Accuracy train: 100.00%
Test Error:
	Avg loss: 0.00020, Accuracy: 99.73%



### Test the model

In [25]:
if TRAIN_SUIT_MODEL:
    test_loop(dl_test_suit, model, criterion)

Test Error:
	Avg loss: 0.00058, Accuracy: 98.64%



### Save the model

In [26]:
if SAVE_SUIT_MODEL:
    models_dir = os.path.join('data', 'models')

    if not os.path.isdir(models_dir):
        os.mkdir(models_dir)
        
    torch.save(model.state_dict(), os.path.join(models_dir, "model_suits.pt"))

In [41]:
### TODO do something with this TODO ###
predict(test_suits, NB_CLASS_SUIT, CNN, "data/models/model_suits.pt")

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3])

### Task 0

### Task 1

### Task 2

---
### 4.1 Example Final results

Example of output you **should** provide the day of the final exam.

In [ ]:
# Creates dummy predictions (toy exmaple)
pred_rank = np.array(["0D"]*4*13).reshape((13, 4)) # Everyone played the "0 of spade".
pred_dealer = [1]*13                # List of players selected as dealer for each round
pred_pts_stand = [0,0,0,13]         # Player 4 won 13 points with standard rules.
pred_pts_advan = [0,0,8,7]          # Player 3 and 4 won 8 and 7 points with adv, rules respectively.

print_results(
    rank_colour=pred_rank, 
    dealer=pred_dealer, 
    pts_standard=pred_pts_stand,
    pts_advanced=pred_pts_advan,
)

---
### 4.2 Example Accuracy

Example of code you can use to validate the performance of your model. Be careful the day of the exam you will not have access to the ground truth of the predictions.

In [ ]:
# Load ground truth from game 1
cgt = pd.read_csv('train_games/game1/game1.csv', index_col=0)
cgt_rank = cgt[['P1', 'P2', 'P3', 'P4']].values

# Compute accuracy of prediction
acc_standard = evaluate_game(pred_rank, cgt_rank, mode_advanced=False)
acc_advanced = evaluate_game(pred_rank, cgt_rank, mode_advanced=True)
print("Your model accuracy is: Standard={:.3f}, Advanced={:.3f}".format(acc_standard, acc_advanced))